# Initialisation

## Imports

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoModelForCausalLM , AutoTokenizer, pipeline, RobertaTokenizer, RobertaForMaskedLM, AlbertTokenizer, AlbertModel, AlbertForMaskedLM
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from afinn import Afinn
from tqdm import tqdm
import unidecode
from time import sleep
from googleapiclient import discovery
import json
from googleapiclient import discovery
import json
import asyncio
import time
import math 
logging.basicConfig(level=logging.INFO)# OPTIONAL

## MAC Settings

In [ ]:
print(f"PyTorch version: {torch.__version__}")
# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

## Definitions

In [ ]:
#TEMPLATES
#TEMPLATES
EVALUATION_PATH = '../data/evaluation/'
RESULTS_PATH = '../data/results/'
ALBERT_BASE_TEMPLATE_1 = 'albert-base-v2_template_1.csv'
ALBERT_BASE_TEMPLATE_5 = 'albert-base-v2_template_5.csv'
ALBERT_LARGE_TEMPLATE_1 = 'albert-large-v2_template_1.csv'
ALBERT_LARGE_TEMPLATE_5 = 'albert-large-v2_template_5.csv'
BERT_BASE_TEMPLATE_1 = 'bert-base-uncased_template_1.csv'
BERT_BASE_TEMPLATE_5 = 'bert-base-uncased_template_5.csv'
BERT_LARGE_TEMPLATE_1 = 'bert-large-uncased_template_1.csv'
BERT_LARGE_TEMPLATE_5 = 'bert-large-uncased_template_5.csv'
ROBERTA_BASE_TEMPLATE_1 = 'roberta-base_template_1.csv'
ROBERTA_BASE_TEMPLATE_5 = 'roberta-base_template_5.csv'
ROBERTA_LARGE_TEMPLATE_1 = 'roberta-large_template_1.csv'
ROBERTA_LARGE_TEMPLATE_5 = 'roberta-large_template_5.csv'
BERTWEET_BASE_TEMPLATE_1 = 'vinai/bertweet-base_template_1.csv'
BERTWEET_BASE_TEMPLATE_5 = 'vinai/bertweet-base_template_5.csv'
BERTWEET_LARGE_TEMPLATE_1 = 'vinai/bertweet-large_template_1.csv'
BERTWEET_LARGE_TEMPLATE_5 = 'vinai/bertweet-large_template_5.csv'



#KEY
QUEERNESS = 'queerness'
PRONOUN = 'pronoun'
TYPE = 'type'
BINARY = 'binary'
NEUTRAL = 'neutral'
NEO = 'neo'
SENT = 'sentiment_analysis'
QUEER = 'queer'
IDENTITY = 'identity'
OTHER = 'other'
ORIENTATION = 'orientation'
HURTLEX = 'hurtlex'
PERSPECTIVE = 'perspective'

perspective_cat = ['Toxicity', 'Identity attack', 'insult', 'Profanity', 'Threat']
hurtlex_cat = ['Animals', 'Crime and immoral behavior', 'Derogatory words', 'Female genitalia', 'Male genitalia', 'Homosexuality', 'Potential negative connotations', 'Professions and occupations', 'Prostitution', 'Social and economic disadvantage']
hurtlex_cat_names = ['an', 're', 'cds', 'asf', 'asm', 'om', 'qas', 'pa', 'pr', 'is']

# Queer Bench - Results

### Utils

In [ ]:
def get_modelName(template_path):
    res = re.sub('_', '', template_path)
    res = re.sub('.csv', '', res)
    res = re.sub('template', '', res)
    att = res[-1]
    res = res[:-1]
    return res, att

def sentiment_score(input_numbers):
    #media_distanza = sum(abs(x) for x in input_numbers) / len(input_numbers)
    #punteggio = 50 - media_distanza * 20
    #return punteggio
    somma = sum(input_numbers)
    media = somma / len(input_numbers)
    output = int(media * 10)
    print(media)
    
# def sentiment_analysis(array, val):
#     if val == 0:
#         array[1] = array[1]+1
#     else:
#         if val < 0: 
#             array[0] = array[0]+1
#         else:
#             array[2] = array[2]+1
#     return array

def checkNaN(str):
  return str != str

def format_array(prediction):
    list = []
    if checkNaN(prediction):
        return list
    else:
        prediction = re.sub("\'", "", prediction)
        prediction = re.sub(" ", "", prediction)
        prediction = re.sub("\[", "", prediction)
        prediction = re.sub("\]", "", prediction)
        list = prediction.split(',')
        return list

def hurtlex(store, array):
    array = format_array(array)
    for val in array:
        for ind, hurt in enumerate(hurtlex_cat_names):
            if val == hurt:
                store[ind] = store[ind]+1
    return store

def perspective(store, array):
    array = format_array(array)
    return [x + y for x, y in zip(store, list(map(int, array)))]

def truncate_float(float_number, decimal_places):
    multiplier = 10 ** decimal_places
    return int(float_number * multiplier) / multiplier


In [ ]:
class QueerResults():
    def __init__(self, template_path):
        self.template_path = template_path
        self.template_file = pd.read_csv(EVALUATION_PATH+template_path, sep=";")
        self.model_name, self.numAtt = get_modelName(template_path)
        self.sentiment_analysis_graph()
        self.hurtlex_graph()
        #self.perspective_graph()

    def sentiment_analysis_graph(self):
        neo, neutral, binary, orientation, identity, other, orientation_queer, identity_queer, other_queer, queer, non_queer = [] ,[] ,[],[],[],[],[],[],[],[],[]
        for index,row in tqdm(self.template_file.iterrows(), total=self.template_file.shape[0], desc='Reading Sentiment Analysis', unit='sentences'):
            #scorro tutti i pronomi 
            val = row.loc[SENT]
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo.append(val)
                elif row.loc[TYPE]== NEUTRAL:
                        neutral.append(val)
                else:
                    binary.append(val)
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    queer.append(val)
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer.append(val)
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation_queer.append(val)
                    else:
                        other_queer.append(val)
                else:
                    non_queer.append(val)
                    if row.loc[TYPE] == IDENTITY:
                        identity.append(val)
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation.append(val)
                    else:
                        other.append(val)


        df = pd.DataFrame(columns=['Score'])
        df.loc['Neo'] =                     truncate_float(sentiment_score(neo),2)        
        df.loc['Neutral'] =                 truncate_float(sentiment_score(neutral),2)
        df.loc['Binary'] =                  truncate_float(sentiment_score(binary),2)
        df.loc['Queer Identity'] =          truncate_float(sentiment_score(identity_queer),2)
        df.loc['Queer Orientation'] =       truncate_float(sentiment_score(orientation_queer),2)
        df.loc['Queer Other'] =             truncate_float(sentiment_score(other_queer),2)
        df.loc['Non Queer Identity'] =      truncate_float(sentiment_score(identity),2)
        df.loc['Non Queer Orientation'] =   truncate_float(sentiment_score(orientation),2)
        df.loc['Non Queer Other'] =         truncate_float(sentiment_score(other),2)
        df.loc['Queer'] =   truncate_float(sentiment_score(queer),2)
        df.loc['Non Queer'] =         truncate_float(sentiment_score(non_queer),2)
        display(df)
        #df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_sentiment.csv', sep=';', index=False)

    
    def hurtlex_graph(self):
        neo, neutral, binary, orientation, identity, other, orientation_queer, identity_queer, other_queer, queer, non_queer = [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] , [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0] , [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0], [0, 0, 0, 0 , 0, 0, 0, 0, 0, 0]
        for index,row in self.template_file.iterrows():
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = hurtlex(neo, row.loc[HURTLEX])
                elif row.loc[TYPE]== NEUTRAL:
                    neutral = hurtlex(neutral, row.loc[HURTLEX])
                else:
                    binary = hurtlex(binary, row.loc[HURTLEX])
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    queer = hurtlex(queer, row.loc[HURTLEX])
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = hurtlex(identity_queer, row.loc[HURTLEX])
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation_queer = hurtlex(orientation_queer, row.loc[HURTLEX])
                    else:
                        other_queer = hurtlex(other_queer, row.loc[HURTLEX])
                else:
                    non_queer = hurtlex(non_queer, row.loc[HURTLEX])
                    if row.loc[TYPE] == IDENTITY:
                        identity = hurtlex(identity, row.loc[HURTLEX])
                    elif row.loc[TYPE] == ORIENTATION:
                        orientation = hurtlex(orientation, row.loc[HURTLEX])
                    else:
                        other = hurtlex(other, row.loc[HURTLEX])
        df = pd.DataFrame(columns=hurtlex_cat)
        df.loc['Neo'] = neo
        df.loc['Neutral'] = neutral
        df.loc['Binary'] = binary
        df.loc['Queer Identity'] = identity_queer
        df.loc['Queer Orientation'] = orientation_queer
        df.loc['Queer Other'] = other_queer
        df.loc['Non Queer Identity'] = identity
        df.loc['Non Queer Orientation'] = orientation
        df.loc['Non Queer Other'] = other
        df.loc['Queer'] = queer
        df.loc['Non Queer'] = non_queer
        display(df)
        #df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_hurtlex.csv', sep=';', index=False)

    def perspective_graph(self):
        neo               = [0, 0, 0, 0, 0] 
        neutral           = [0, 0, 0, 0, 0] 
        binary            = [0, 0, 0, 0, 0]
        orientation       = [0, 0, 0, 0, 0]
        identity          = [0, 0, 0, 0, 0]
        other             = [0, 0, 0, 0, 0]
        orientation_queer = [0, 0, 0, 0, 0]
        identity_queer    = [0, 0, 0, 0, 0]
        other_queer       = [0, 0, 0, 0, 0]
        for index,row in self.template_file.iterrows():
            #scorro tutti i pronomi 
            if row.loc[QUEERNESS] == PRONOUN:
                if row.loc[TYPE]== NEO:
                    neo = perspective(neo, row.loc[PERSPECTIVE])
                else:
                    if row.loc[TYPE]== NEUTRAL:
                        neutral = perspective(neutral, row.loc[PERSPECTIVE])
                    else:
                        binary = perspective(binary, row.loc[PERSPECTIVE])
            #scorro tutti i termini
            else:
                if row.loc[QUEERNESS] == QUEER:
                    if row.loc[TYPE] == IDENTITY:
                        identity_queer = perspective(identity_queer, row.loc[PERSPECTIVE])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation_queer = perspective(orientation_queer, row.loc[PERSPECTIVE])
                        else:
                            other_queer = perspective(other_queer, row.loc[PERSPECTIVE])
                else:
                    if row.loc[TYPE] == IDENTITY:
                        identity = perspective(identity, row.loc[PERSPECTIVE])
                    else:
                        if row.loc[TYPE] == ORIENTATION:
                            orientation = perspective(orientation, row.loc[PERSPECTIVE])
                        else:
                            other = perspective(other, row.loc[PERSPECTIVE])
        df = pd.DataFrame(columns=perspective_cat)
        df.loc['Neo'] = neo
        df.loc['Neutral'] = neutral
        df.loc['Binary'] = binary
        df.loc['Queer Identity'] = identity_queer
        df.loc['Queer Orientation'] = orientation_queer
        df.loc['Queer Other'] = other_queer
        df.loc['Non Queer Identity'] = identity
        df.loc['Non Queer Orientation'] = orientation
        df.loc['Non Queer Other'] = other
        display(df)
        df.to_csv(RESULTS_PATH+self.model_name+'_'+self.numAtt+'_perspective.csv', sep=';', index=False)


out: va da -50 a 50 a -50

In [205]:
def sentiment_score(input_numbers):
    somma = sum(input_numbers)
    media = somma / len(input_numbers)
    output = int(media * 10)
    print(output)

# Esempio di utilizzo
input_numero = [5,3, 5, 3, 5]  # Sostituisci con il tuo input desiderato tra -50 e 50
risultato = sentiment_score(input_numero)
#print(risultato)

42


in: num da -50 a 0 a -50
out: num da 100 a 0

In [211]:
def sentiment_score(input_numbers):
    media_distanza = abs(input_numbers/10)
    print(media_distanza)
    punteggio = 50 - media_distanza * 20
    return int(punteggio)

# Esempio di utilizzo
input_numero = 0  # Sostituisci con il tuo input desiderato tra -50 e 50
risultato = sentiment_score(input_numero)
print(risultato)

0.0
50


In [219]:
# sent analysis trasformation

def trasforma_numero(numero):
    # Calcolare la distanza del numero da zero
    distanza_da_zero = abs(numero)
    
    # Mappare la distanza da zero al range desiderato (0-100)
    output = 100 - (distanza_da_zero * 2)
    
    # Assicurarsi che l'output sia compreso tra 0 e 100
    #output = max(0, min(100, output))
    
    # Calcola il valore inverso
    output = 100 - output
    
    return output

# Test della funzione con alcuni valori di input
numero_input = 0
output = trasforma_numero(numero_input)
print(f"Input: {numero_input}, Output: {output}")

numero_input = -20
output = trasforma_numero(numero_input)
print(f"Input: {numero_input}, Output: {output}")

numero_input = 50
output = trasforma_numero(numero_input)
print(f"Input: {numero_input}, Output: {output}")


Input: 0, Output: 0
Input: -20, Output: 40
Input: 50, Output: 100
